In [1]:
import geopandas as gpd
import pandas as pd
import networkx as nx
import osmnx as ox
import folium
import pickle
from shapely.geometry import box, Point
from geopy.distance import geodesic
import json

### Carregando o Grafo

In [2]:
# Ler o grafo de um arquivo usando pickle
with open('grafoDirecional.grafo', 'rb') as f:
    G = pickle.load(f)

In [3]:
print(len(G.nodes))
print(len(G.edges))

4544
8166


In [4]:
print(G.nodes(data=True))
print(G.edges(data=True))

[(6856, {'coords': <POINT (-47.785 -15.763)>}), (7556, {'coords': <POINT (-47.789 -15.764)>}), (7558, {'coords': <POINT (-47.789 -15.77)>}), (7559, {'coords': <POINT (-47.792 -15.768)>}), (7560, {'coords': <POINT (-47.794 -15.764)>}), (7562, {'coords': <POINT (-47.789 -15.763)>}), (7563, {'coords': <POINT (-47.787 -15.763)>}), (6844, {'coords': <POINT (-47.786 -15.764)>}), (6134, {'coords': <POINT (-47.781 -15.779)>}), (2715, {'coords': <POINT (-47.779 -15.782)>}), (2397, {'coords': <POINT (-47.795 -15.803)>}), (2398, {'coords': <POINT (-47.801 -15.809)>}), (2399, {'coords': <POINT (-47.806 -15.813)>}), (2400, {'coords': <POINT (-47.806 -15.818)>}), (2401, {'coords': <POINT (-47.805 -15.821)>}), (2402, {'coords': <POINT (-47.807 -15.825)>}), (2403, {'coords': <POINT (-47.811 -15.829)>}), (2404, {'coords': <POINT (-47.814 -15.832)>}), (2405, {'coords': <POINT (-47.822 -15.835)>}), (2441, {'coords': <POINT (-47.826 -15.831)>}), (2535, {'coords': <POINT (-47.833 -15.819)>}), (3617, {'coor

### Encontrar as paradas mais próximos em um raio de distância

In [3]:
raio = 0.007  # Raio desejado em metros ???

origem = (-15.989444964529529, -48.044418962814866) # Unb Gama
destino = (-15.79113644987054, -47.88317800514907)   # Conjunto Nacional
destino2 = (-15.864892647091217, -48.030234362882624) # Universidade Catolica


def calculaNoProximo(G, long, lat):
    distancias = {}
    ponto = Point(long, lat)
    for node in G.nodes():
        node_coords = Point(G.nodes[node]['coords'])  
        dist = node_coords.distance(ponto)
        distancias[node] = {'dist': dist}
    # Find the nodes with the shortest distancias to the origin and destination
    noMaisPerto = min(distancias, key=lambda x: distancias[x]['dist'])
    return noMaisPerto


def calculaNosProximos(G, long, lat, raio):
    nos_proximos = []
    ponto = Point(long, lat)
    for node in G.nodes():
        node_coords = Point(G.nodes[node]['coords'])
        dist = node_coords.distance(ponto)
        if dist <= raio:
            nos_proximos.append(node)
    return nos_proximos

nos_origem = calculaNosProximos(G, origem[1], origem[0], raio)
no_destino = calculaNoProximo(G, destino2[1], destino2[0])

# Mapa folium
m = folium.Map(location=[origem[0], origem[1]], zoom_start=14)

# Marcadores para os nós próximos de origem
for node in nos_origem:
    coords = G.nodes[node]['coords']
    folium.Marker([coords.y, coords.x], icon=folium.Icon(color='blue')).add_to(m)

# Marcadores para os nós próximos de destino
coords = G.nodes[no_destino]['coords']
folium.Marker([coords.y, coords.x], icon=folium.Icon(color='red')).add_to(m)

# Exiba o mapa
m

In [53]:
print(nos_origem)

[7024, 7027, 1855, 1856, 7672, 7025, 7026]


### Algoritmo rota ótima

- Deve receber o nó origem e o nó destino do Grafo
- Percorrer o Grafo usando algoritmo de Dijkstra, cujo o peso é a distancia entre as arestas
- Armazenar as Paradas percorridas e as linhas de ônibus necessárias para chegar da origem ao destino

In [58]:
# Calcula rota otima com algoritmo de Dijkstra
paradas = nx.shortest_path(G, nos_origem[4], no_destino, weight="dist", method="dijkstra")

print(paradas)

[7672, 2023, 7021, 7020, 7015, 7014, 7010, 7006, 2620, 3369, 3464, 3480, 3482]


### Utilizando Dijkstra para adicionar peso nas baldeações

- Caso o algoritmo troque de linha de onibus (altere de um nó que seguia uma linha de onibus X, para um nó que segue linha de ônibus Y)
- Então a distância entre os nós deve ser multiplicada por um *PESO*, para dificultar a baldeação de linhas

In [5]:
import heapq

# Peso (PESO) para dificultar a baldeação das linhas ônibus e aplicar no algoritmo.
PESO = 10

RAIO_PARADAS_PROXIMAS = 0.009

# Função para calcular a distância euclidiana entre dois pontos
def distancia_entre_nos(grafo, no_origem, no_destino):
    origem = (grafo.nodes[no_origem]['coords'].y, grafo.nodes[no_origem]['coords'].x)
    destino = (grafo.nodes[no_destino]['coords'].y, grafo.nodes[no_destino]['coords'].x)
    return geodesic(origem, destino).km

def dijkstra(grafo, origem, destino, raio):
    distancias = {node: float('inf') for node in grafo.nodes}
    distancias[origem] = 0 
    predecessores = {}
    linhas = {}

    # Fila prioridade com: Distancia até o nó destino, Linha de Ônibus Atual, Nó atual
    fila_prioridade = [(0, None, origem)]

    while fila_prioridade:

        distancia_atual, linha_atual, no_atual = heapq.heappop(fila_prioridade)

        if no_atual == destino or (distancia_entre_nos(grafo, no_atual, destino) <= raio):
            caminho = []
            linhas_caminho = []
            while no_atual is not None:
                caminho.append(no_atual)
                linhas_caminho.append(linhas.get(no_atual))
                no_atual = predecessores.get(no_atual)
            return list(reversed(caminho)), list(reversed(linhas_caminho))

        if distancia_atual > distancias[no_atual]:
            continue

        for vizinho in grafo.neighbors(no_atual):

            linha_atual_aux = linha_atual
            dist_atual_aux = distancia_atual
            arestas = grafo.get_edge_data(no_atual, vizinho)

            if distancia_entre_nos(grafo, no_atual, vizinho) <= RAIO_PARADAS_PROXIMAS: 
                distancias[vizinho] = dist_atual_aux
                linhas[vizinho] = linha_atual_aux
                predecessores[vizinho] = no_atual
                heapq.heappush(fila_prioridade, (dist_atual_aux, linha_atual_aux, vizinho))
                continue
            
            if linha_atual_aux is None:
                dist_atual_aux += arestas['dist']
                linha_atual_aux = arestas['linha'][0]
            elif linha_atual_aux in arestas['linha']:
                dist_atual_aux += arestas['dist']
            else:
                dist_atual_aux += PESO * arestas['dist']
                linha_atual_aux = arestas['linha'][0]
        
            if distancia_atual < distancias[vizinho]:
                distancias[vizinho] = dist_atual_aux
                linhas[vizinho] = linha_atual_aux
                predecessores[vizinho] = no_atual
                heapq.heappush(fila_prioridade, (dist_atual_aux, linha_atual_aux, vizinho))

    return [], []

raio = 0.500

linhas_otimas = []
rotas_otimas = []

# for no_origem in nos_origem:
#     paradas, linhas_usadas = dijkstra(G, nos_origem[4], no_destino, raio)
#     rotas_otima.append(dijkstra(G, nos_origem[4], no_destino, raio))
#     linhas_otimas.append(linhas_usadas)

paradas, linhas_usadas = dijkstra(G, nos_origem[4], no_destino, raio)

In [5]:
print(paradas)
print(linhas_usadas)

[7672, 7907, 7908, 6350, 3809, 3810, 6354, 7909, 3811, 7910, 3813, 7911, 7912, 3814, 7913, 3816, 7914, 7915, 3817, 7917, 7922, 7923, 5391, 4975, 4976]
[None, '0.205', '0.205', '0.205', '0.255', '0.205', '0.205', '0.205', '0.205', '0.205', '0.205', '0.205', '0.255', '0.255', '0.255', '0.255', '0.255', '0.205', '0.205', '0.255', '0.205', '0.205', '0.205', '0.205', '0.205']


### Identifica todas as linhas usadas, sem duplicação

In [6]:
linhas = []
for linha in linhas_usadas[1:]:
    if linha not in linhas:
        linhas.append(linha)

print(linhas)

['0.205', '0.255']


### Identifica qual rota tem menos linhas de ônibus

In [19]:
todasLinhas = []
for rota in linhas_otimas:
    linhas = []
    for linha in rota[1:]:
        if linha not in linhas:
            linhas.append(linha)
    todasLinhas.append(linhas)

indice, linhaMenosOnibus = min(enumerate(todasLinhas), key=lambda x: len(x[1]))

print(linhaMenosOnibus)
print(rotas_otima[indice])

['865.2', '0.882', '807.8', '805.7', '0.205']
[7672, 1936, 7908, 6350, 3809, 3810, 6354, 7909, 3811, 7910, 6873, 6872, 6355, 6352, 6874, 6875, 6353, 6870, 6997, 7637, 4000, 7901, 7899, 7900, 4001, 4002, 7898, 4003, 4004, 4005, 4006, 4007, 4008, 4009, 4010, 4011, 4012, 4013, 4014, 4015, 6694, 5391, 4975, 4976]


In [23]:
# # Inicializar o mapa do Folium (ajuste as coordenadas iniciais conforme necessário)
m = folium.Map(location=[-15.7942, -47.8825], zoom_start=12)

linhas = []

# Cores para os grafos
cores = ['blue', 'red']
contador_cor = 0
cor = cores[contador_cor] 

# Adicionando primeira parada no Folium
lat_origem, lon_origem = G.nodes[paradas[0]]['coords'].y, G.nodes[paradas[0]]['coords'].x
folium.Marker([lat_origem, lon_origem], popup=f'Parada {paradas[0]}').add_to(m)

# Primeira linha usada
linha_atual = linhas_usadas[1]

for i in range(1, len(paradas)):

    linha_aux = linhas_usadas[i]

    parada_origem = paradas[i-1] 
    parada_destino = paradas[i]

    # Adicionando paradas no Folium
    
    lat_dest, lon_dest = G.nodes[parada_destino]['coords'].y, G.nodes[parada_destino]['coords'].x
    folium.Marker([lat_dest, lon_dest], popup=f'Parada {parada_destino}').add_to(m)

    # Adicionando arestas no Folium
    origem_coords = (G.nodes[parada_origem]['coords'].y, G.nodes[parada_origem]['coords'].x)
    destino_coords = (lat_dest, lon_dest)

    # Adicionando nova linha na lista
    if linha_atual is not linha_aux:
        linha_atual = linha_aux
        contador_cor += 1
        cor = cores[contador_cor % len(cores)]

    folium.PolyLine([origem_coords, destino_coords], tooltip=linha_atual, color=cor, weight=2).add_to(m)

        
m

In [32]:
print(list(G.neighbors(3817)))
print(G[3809][3810])

[7917, 7870]
{'dist': 0.37799632278283074, 'linha': ['0.205', '0.234', '0.255', '255.2', '0.275', '865.1', '260.3', '0.273', '205.1', '0.253', '0.225', '815.1', '865.2']}


In [29]:
# # Inicializar o mapa do Folium (ajuste as coordenadas iniciais conforme necessário)
m = folium.Map(location=[-15.7942, -47.8825], zoom_start=12)

LINHA_ALVO = str(0.255)

alvos = [2024,7907,2023,1936]

for no1, no2, data in G.edges(data=True):

    if no2 in alvos:
        lat2, lon2 = G.nodes[no2]['coords'].y, G.nodes[no2]['coords'].x
        folium.Marker([lat2, lon2], popup=f'Parada {no2}').add_to(m)

    if LINHA_ALVO in data['linha']:

        lat1, lon1 = G.nodes[no1]['coords'].y, G.nodes[no1]['coords'].x
        folium.Marker([lat1, lon1], popup=f'Parada {no1}').add_to(m)
        
        lat2, lon2 = G.nodes[no2]['coords'].y, G.nodes[no2]['coords'].x
        folium.Marker([lat2, lon2], popup=f'Parada {no2}').add_to(m)

        # Adicionando arestas no Folium
        origem_coords = (lat1, lon1)
        destino_coords = (lat2, lon2)
        folium.PolyLine([origem_coords, destino_coords], tooltip=f"{no1} ate {no2}", color='black', weight=2).add_to(m)

m

In [30]:
# # Inicializar o mapa do Folium (ajuste as coordenadas iniciais conforme necessário)
m = folium.Map(location=[-15.7942, -47.8825], zoom_start=12)

LINHA_ALVO = str(205.1)

alvos = [2024,7907,2023,1936]

for no1, no2, data in G.edges(data=True):

    if no2 in alvos:
        lat2, lon2 = G.nodes[no2]['coords'].y, G.nodes[no2]['coords'].x
        folium.Marker([lat2, lon2], popup=f'Parada {no2}').add_to(m)

    if LINHA_ALVO in data['linha']:

        lat1, lon1 = G.nodes[no1]['coords'].y, G.nodes[no1]['coords'].x
        folium.Marker([lat1, lon1], popup=f'Parada {no1}').add_to(m)
        
        lat2, lon2 = G.nodes[no2]['coords'].y, G.nodes[no2]['coords'].x
        folium.Marker([lat2, lon2], popup=f'Parada {no2}').add_to(m)

        # Adicionando arestas no Folium
        origem_coords = (lat1, lon1)
        destino_coords = (lat2, lon2)
        folium.PolyLine([origem_coords, destino_coords], tooltip=f"{no1} ate {no2}", color='black', weight=2).add_to(m)

m

In [ ]:
# # Inicializar o mapa do Folium (ajuste as coordenadas iniciais conforme necessário)
m = folium.Map(location=[-15.7942, -47.8825], zoom_start=12)

linhas = []

paradas = rotas_otima[indice]

# Cores para os grafos
cores = ['blue', 'red', 'yellow', 'green']
contador_cor = 0
cor = cores[contador_cor] 

# Adicionando primeira parada no Folium
lat, lon = G.nodes[paradas[1][0]]['coords'].y, G.nodes[paradas[1][0]]['coords'].x
folium.Marker([lat, lon], popup=f'Parada {paradas[0]}').add_to(m)

for i in range(2, len(paradas)):

    parada_origem = paradas[i-1][0] 
    parada_destino = paradas[i][0]

    # Adicionando paradas no Folium
    lat2, lon2 = G.nodes[parada_destino]['coords'].y, G.nodes[parada_destino]['coords'].x
    folium.Marker([lat2, lon2], popup=f'Parada {parada_destino}').add_to(m)

    # Adicionando arestas no Folium
    origem_coords = (G.nodes[parada_origem]['coords'].y, G.nodes[parada_origem]['coords'].x)
    destino_coords = (lat2, lon2)
    dados_aresta = G.get_edge_data(parada_origem, parada_destino)

    # Adicionando nova linha na lista
    if dados_aresta['linha'] not in linhas:
        linhas.append(dados_aresta['linha'])
        contador_cor += 1
        cor = cores[contador_cor % len(cores)]

    folium.PolyLine([origem_coords, destino_coords], tooltip=dados_aresta['linha'], color=cor, weight=2).add_to(m)

        
print(linhas)

m

In [63]:
# # Inicializar o mapa do Folium (ajuste as coordenadas iniciais conforme necessário)
m = folium.Map(location=[-15.7942, -47.8825], zoom_start=12)

linhas = []

# Cores para os grafos
cores = ['blue', 'red', 'yellow', 'green']
contador_cor = 0
cor = cores[contador_cor] 

# Adicionando primeira parada no Folium
lat_origem, lon_origem = G.nodes[paradas[0]]['coords'].y, G.nodes[paradas[0]]['coords'].x
folium.Marker([lat_origem, lon_origem], popup=f'Parada {paradas[0]}').add_to(m)

linha_aux = None

for i in range(1, len(paradas)):

    parada_origem = paradas[i-1] 
    parada_destino = paradas[i]

    # Adicionando paradas no Folium
    
    lat_dest, lon_dest = G.nodes[parada_destino]['coords'].y, G.nodes[parada_destino]['coords'].x
    folium.Marker([lat_dest, lon_dest], popup=f'Parada {parada_destino}').add_to(m)

    # Adicionando arestas no Folium
    origem_coords = (G.nodes[parada_origem]['coords'].y, G.nodes[parada_origem]['coords'].x)
    destino_coords = (lat_dest, lon_dest)

    data = G.get_edge_data(parada_origem, parada_destino)


    # Adicionando nova linha na lista
    if linha_aux is not data['linha'] :
        linha_aux = data['linha']
        contador_cor += 1
        cor = cores[contador_cor % len(cores)]

    folium.PolyLine([origem_coords, destino_coords], tooltip=linha_aux, color=cor, weight=2).add_to(m)

        
m

### Retornar as paradas e linhas de ônibus que levam da origem ao destino, em arquivo json 

In [7]:
# Considere que 'paradas' são todas os nós de paradas de ônibus usados na rota otima, e 'linhas_usadas' todas as linhas de ônibus usadas na rota ótima
paradas, linhas_usadas = dijkstra(G, nos_origem[4], no_destino, raio)

paradas_info = []
linhas_usadas = linhas_usadas[1:]

# Itera sobre as paradas e linhas para criar o dicionário
for i, parada in enumerate(paradas[1:]):
    parada_info = {
        "id_stop": int(parada),
        "lat": float(G.nodes[parada]['coords'].y),
        "lon": float(G.nodes[parada]['coords'].x),
        "linha": str(linhas_usadas[i])
    }
    paradas_info.append(parada_info)


# Salve o dicionário em um arquivo JSON
with open("paradas_info.json", "w") as json_file:
    json.dump(paradas_info, json_file, indent=4)
